# ***Sistema Experto para Árboles Genealógicos***

In [5]:
pip install experta

  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3149 sha256=d7fbf71b51b98a2870e13e9f6d05de41e25095ddd29769a7d4ca1e09f741afad
  Stored in directory: /root/.cache/pip/wheels/f6/ff/aa/750fec7bf9618d87b53572def5abf3e098f853cc5ab4147656
Successfully built frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.6
    Uninstalling frozendict-2.4.6:
      Successfully uninstalled frozendict-2.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.65 requires frozendict>=2.3.4, but you have frozendict 1.2 which is incompatible.


In [6]:
import collections.abc
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping

In [8]:
from experta import *

# === Definición de hechos base ===
class Hombre(Fact):
    """Representa a un hombre en la familia"""
    pass

class Mujer(Fact):
    """Representa a una mujer en la familia"""
    pass

class Padre(Fact):
    """Relación de paternidad"""
    pass

class Madre(Fact):
    """Relación de maternidad"""
    pass

# === Hechos derivados por inferencia ===
class Progenitor(Fact):
    """Padre o madre"""
    pass

class Abuelo(Fact):
    """Abuelo de alguien"""
    pass

class Abuela(Fact):
    """Abuela de alguien"""
    pass

class Hermano(Fact):
    """Relación entre hermanos hombres"""
    pass

class Hermana(Fact):
    """Relación entre hermanas mujeres"""
    pass

class Tio(Fact):
    """Relación de tío con sobrino"""
    pass

class Tia(Fact):
    """Relación de tía con sobrino"""
    pass

class Primo(Fact):
    """Relación entre primos"""
    pass

# === Motor de inferencia basado en reglas ===
class ArbolGenealogico(KnowledgeEngine):

    def __init__(self):
        super().__init__()
        # Conjunto para llevar control de relaciones procesadas
        self.relaciones_procesadas = set()

    # === REGLAS DE INFERENCIA COMPLETAS (EJEMPLOS) ===

    #
    # Padre
    #
    @Rule(Padre(padre=MATCH.padre, hijo=MATCH.hijo), salience=100)
    def padre_a_progenitor(self, padre, hijo):
        self.declare(Progenitor(progenitor=padre, hijo=hijo))
        print(f"Progenitor: {padre} es progenitor de {hijo}")
    #
    # Madre
    #
    @Rule(Madre(madre=MATCH.madre, hijo=MATCH.hijo), salience=100)
    def madre_a_progenitor(self, madre, hijo):
        self.declare(Progenitor(progenitor=madre, hijo=hijo))
        print(f"Progenitor: {madre} es progenitor de {hijo}")

    # === REGLAS CON ESPACIOS PARA COMPLETAR ===

    #
    # Abuelo
    #
    @Rule(Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo),
          Progenitor(progenitor=MATCH.hijo, hijo=MATCH.nieto),
          Hombre(nombre=MATCH.progenitor), salience=80)
    def inferir_abuelo(self, progenitor, nieto):
        self.declare(Abuelo(abuelo=progenitor, nieto=nieto))
        print(f"Abuelo: {progenitor} es abuelo de {nieto}")
    #
    # Abuela
    #
    @Rule(Progenitor(progenitor=MATCH.progenitor , hijo=MATCH.hijo),
          Progenitor(progenitor=MATCH.hijo, hijo=MATCH.nieto),
          Mujer(nombre=MATCH.progenitor), salience=80)
    def inferir_abuela(self, progenitor, nieto):
        self.declare(Abuela(abuela=progenitor, nieto=nieto))
        print(f"Abuela: {progenitor} es abuela de {nieto}")
    #
    # Hermano
    #
    @Rule(Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo1),
          Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo2),
          Hombre(nombre=MATCH.hijo1), salience=60)
    def inferir_hermano(self, progenitor, hijo1, hijo2):
        if hijo1 == hijo2:
            return
        relacion_id = f"hermano_{hijo1}_{hijo2}"
        if relacion_id in self.relaciones_procesadas:
            return
        self.relaciones_procesadas.add(relacion_id)
        self.declare(Hermano(hermano=hijo1, hermano_de=hijo2))
        print(f"Hermano: {hijo1} es hermano de {hijo2}")
    #
    # Hermana
    #
    @Rule(Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo1),
          Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo2),
          Mujer(nombre=MATCH.hijo1), salience=60)
    def inferir_hermana(self, progenitor, hijo1, hijo2):
        if hijo1 == hijo2:
            return
        relacion_id = f"hermana_{hijo1}_{hijo2}"
        if relacion_id in self.relaciones_procesadas:
            return
        self.relaciones_procesadas.add(relacion_id)
        self.declare(Hermana(hermana=hijo1, hermana_de=hijo2))
        print(f"Hermana: {hijo1} es hermana de {hijo2}")
    #
    # Tío
    #
    @Rule(Hermano(hermano=MATCH.hermano, hermano_de=MATCH.progenitor),
          Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.sobrino), salience=40)
    def inferir_tio(self, hermano, sobrino):
        self.declare(Tio(tio=hermano, sobrino=sobrino))
        print(f"Tío: {hermano} es tío de {sobrino}")
    #
    # Tía
    #
    @Rule(Hermana(hermana=MATCH.hermana, hermana_de=MATCH.progenitor),
          Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.sobrino), salience=40)
    def inferir_tia(self, hermana, sobrino):
        self.declare(Tia(tia=hermana, sobrino=sobrino))
        print(f"Tía: {hermana} es tía de {sobrino}")
    #
    # Primos (Tío)
    #
    @Rule(Tio(tio=MATCH.tio, sobrino=MATCH.primo1),
          Progenitor(progenitor=MATCH.tio, hijo=MATCH.primo2), salience=20)
    def inferir_primos_por_tio(self, tio, primo1, primo2):
        if primo1 == primo2:
            return
        if primo1 < primo2:
            relacion_id = f"primos_{primo1}_{primo2}"
        else:
            relacion_id = f"primos_{primo2}_{primo1}"
        if relacion_id in self.relaciones_procesadas:
            return
        self.relaciones_procesadas.add(relacion_id)
        self.declare(Primo(primo=primo1, primo_de=primo2))
        self.declare(Primo(primo=primo2, primo_de=primo1))
        print(f"Primos: {primo1} y {primo2} son primos")
    #
    # Primos (Tía)
    #
    @Rule(Tia(tia=MATCH.tia, sobrino=MATCH.primo1),
          Progenitor(progenitor=MATCH.tia, hijo=MATCH.primo2), salience=20)
    def inferir_primos_por_tia(self, tia, primo1, primo2):
        if primo1 == primo2:
            return
        if primo1 < primo2:
            relacion_id = f"primos_{primo1}_{primo2}"
        else:
            relacion_id = f"primos_{primo2}_{primo1}"
        if relacion_id in self.relaciones_procesadas:
            return
        self.relaciones_procesadas.add(relacion_id)
        self.declare(Primo(primo=primo1, primo_de=primo2))
        self.declare(Primo(primo=primo2, primo_de=primo1))
        print(f"Primos: {primo1} y {primo2} son primos")

if __name__ == "__main__":
    # Crear el motor
    motor = ArbolGenealogico()

    # Reiniciar (limpia la memoria antes de cargar hechos)
    motor.reset()

    # === Hombres ===
    motor.declare(Hombre(nombre='juan'))
    motor.declare(Hombre(nombre='pedro'))
    motor.declare(Hombre(nombre='carlos'))
    motor.declare(Hombre(nombre='david'))
    motor.declare(Hombre(nombre='luis'))

    # === Mujeres ===
    motor.declare(Mujer(nombre='maria'))
    motor.declare(Mujer(nombre='ana'))
    motor.declare(Mujer(nombre='sofia'))
    motor.declare(Mujer(nombre='laura'))
    motor.declare(Mujer(nombre='carmen'))

    # === Relaciones padre ===
    motor.declare(Padre(padre='juan', hijo='pedro'))
    motor.declare(Padre(padre='juan', hijo='ana'))
    motor.declare(Padre(padre='pedro', hijo='david'))
    motor.declare(Padre(padre='pedro', hijo='sofia'))
    motor.declare(Padre(padre='carlos', hijo='luis'))
    motor.declare(Padre(padre='carlos', hijo='laura'))

    # === Relaciones madre ===
    motor.declare(Madre(madre='maria', hijo='pedro'))
    motor.declare(Madre(madre='maria', hijo='ana'))
    motor.declare(Madre(madre='laura', hijo='david'))
    motor.declare(Madre(madre='laura', hijo='sofia'))
    motor.declare(Madre(madre='carmen', hijo='luis'))
    motor.declare(Madre(madre='carmen', hijo='laura'))

    # Ejecutar las reglas
    print("\n=== Inferencias del árbol genealógico ===\n")
    motor.run()



=== Inferencias del árbol genealógico ===

Progenitor: carmen es progenitor de laura
Progenitor: carmen es progenitor de luis
Progenitor: laura es progenitor de sofia
Progenitor: laura es progenitor de david
Progenitor: maria es progenitor de ana
Progenitor: maria es progenitor de pedro
Progenitor: carlos es progenitor de laura
Progenitor: carlos es progenitor de luis
Progenitor: pedro es progenitor de sofia
Progenitor: pedro es progenitor de david
Progenitor: juan es progenitor de ana
Progenitor: juan es progenitor de pedro
Abuelo: juan es abuelo de david
Abuelo: juan es abuelo de sofia
Abuela: maria es abuela de david
Abuela: maria es abuela de sofia
Abuelo: carlos es abuelo de david
Abuelo: carlos es abuelo de sofia
Abuela: carmen es abuela de david
Abuela: carmen es abuela de sofia
Hermana: ana es hermana de pedro
Hermano: pedro es hermano de ana
Hermana: sofia es hermana de david
Hermano: david es hermano de sofia
Hermana: laura es hermana de luis
Hermano: luis es hermano de laur

## **Preguntas:**

### **1. ¿Cómo se podría modificar el código para inferir bisabuelos y bisabuelas?**

##### ***Rta:** Creamos una clase y su funcion de la siguiente manera: <br> `class Bisabuelo(Fact):` <br> `"""Bisabuelo de alguien"""` <br> `pass` <br> esa seria la clase y su regla con su función: <br> `@Rule(Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo),` <br> `Progenitor(progenitor=MATCH.hijo, hijo=MATCH.nieto),` <br> `Progenitor(progenitor=MATCH.nieto, hijo=MATCH.bisnieto),` <br> `Hombre(nombre=MATCH.progenitor), salience=70)` <br> `def inferir_bisabuelo(self, progenitor, bisnieto):` <br> `self.declare(Bisabuelo(bisabuelo=progenitor, bisnieto=bisnieto))` <br> ` print(f"Bisabuelo: {progenitor} es bisabuelo de {bisnieto}")`*

### **2. ¿Por qué la inferencia de Abuelo y Abuela requiere dos niveles de Progenitor?**

##### ***Rta:** Porque abuelo ≠ padre directo, es el padre del padre/madre. <br> Por eso necesitamos encadenar dos relaciones de progenitor:*